In [ ]:
import pandas as pd

caminho_arquivo = "../data/03_final/dataset.csv"
df = pd.read_csv(caminho_arquivo)

In [ ]:
df.head()

In [ ]:
df.dtypes

"As métricas que devem constar no relatório são:
- taxa de aumento de casos
- taxa de mortalidade
- taxa de ocupação de UTI
- taxa de vacinação da população

Além das métricas citadas o relatório deve possuir dois gráficos, um representando o número diário de casos dos últimos 30 dias e outro mostrando o número mensal de casos dos últimos 12 meses."

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:
def carregar_e_preparar_dados(caminho_arquivo):
    """
    Carrega os dados do arquivo CSV e converte a coluna de data.

    Args:
        caminho_arquivo (str): O caminho para o arquivo CSV.

    Returns:
        pandas.DataFrame: O DataFrame com a coluna de data convertida.
    """
    df = pd.read_csv(caminho_arquivo)
    # Converte a coluna de data para o formato datetime
    df["DATA_NOTIFICACAO"] = pd.to_datetime(df["DATA_NOTIFICACAO"])
    return df

In [ ]:
def gerar_grafico_casos_diarios(df):
    """
    Gera e salva um gráfico do número de casos diários dos últimos 30 dias.

    Args:
        df (pandas.DataFrame): O DataFrame contendo os dados.
    """
    # Define a data final como a data mais recente no dataframe
    data_final = df["DATA_NOTIFICACAO"].max()
    # Define a data inicial como 30 dias antes da data final
    data_inicial = data_final - pd.Timedelta(days=30)

    # Filtra o dataframe para incluir apenas os últimos 30 dias
    df_ultimos_30_dias = df[df["DATA_NOTIFICACAO"] >= data_inicial]

    # Agrupa por data e conta o número de notificações
    casos_diarios = df_ultimos_30_dias.groupby("DATA_NOTIFICACAO").size()

    # Cria o gráfico
    plt.style.use("seaborn-v0_8-whitegrid")
    plt.figure(figsize=(15, 7))
    plt.plot(casos_diarios.index, casos_diarios.values, marker="o", linestyle="-")

    # Formata o eixo x para mostrar as datas de forma legível
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%d-%m-%Y"))
    plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=2))
    plt.gcf().autofmt_xdate(rotation=45)

    # Adiciona títulos e rótulos
    plt.title("Número de Casos Diários (Últimos 30 Dias)", fontsize=16)
    plt.xlabel("Data da Notificação", fontsize=12)
    plt.ylabel("Número de Casos", fontsize=12)
    plt.tight_layout()

    # Salva a imagem
    plt.savefig("casos_diarios_ultimos_30_dias.png")
    print("Gráfico de casos diários salvo como 'casos_diarios_ultimos_30_dias.png'")

In [ ]:
def gerar_grafico_casos_mensais(df):
    """
    Gera e salva um gráfico do número de casos mensais dos últimos 12 meses.

    Args:
        df (pandas.DataFrame): O DataFrame contendo os dados.
    """
    # Define a data final como o final do mês da data mais recente
    data_final = df["DATA_NOTIFICACAO"].max()
    # Define a data inicial como 12 meses antes da data final
    data_inicial = data_final - pd.DateOffset(months=12)

    # Filtra o dataframe para incluir apenas os últimos 12 meses
    df_ultimos_12_meses = df[df["DATA_NOTIFICACAO"] >= data_inicial]

    # Agrupa por mês (formato Ano-Mês) e conta as notificações
    casos_mensais = df_ultimos_12_meses.groupby(
        df_ultimos_12_meses["DATA_NOTIFICACAO"].dt.to_period("M")
    ).size()

    # Converte o índice do período para string para plotagem
    casos_mensais.index = casos_mensais.index.strftime("%Y-%m")

    # Cria o gráfico
    plt.style.use("seaborn-v0_8-whitegrid")
    plt.figure(figsize=(15, 7))
    casos_mensais.plot(kind="bar", color="skyblue")

    # Adiciona títulos e rótulos
    plt.title("Número de Casos Mensais (Últimos 12 Meses)", fontsize=16)
    plt.xlabel("Mês da Notificação", fontsize=12)
    plt.ylabel("Número de Casos", fontsize=12)
    plt.xticks(rotation=45)
    plt.tight_layout()

    # Salva a imagem
    plt.savefig("casos_mensais_ultimos_12_meses.png")
    print("Gráfico de casos mensais salvo como 'casos_mensais_ultimos_12_meses.png'")

In [ ]:
def calcular_taxa_aumento_casos(df: pd.DataFrame) -> pd.Series:
    """
    Calcula a taxa de aumento diário de casos.

    Args:
        df: O DataFrame do pandas com os dados dos casos.

    Returns:
        Uma série do pandas com a taxa de aumento diário de casos.
    """
    casos_diarios = df.groupby("DATA_NOTIFICACAO").size()
    taxa_aumento = casos_diarios.pct_change() * 100
    return taxa_aumento

    # posso pegar os ultimos dias e cada dia calcular quanto que aumentou em relação ao dia anterior em porcentagem
    # Se um dia ta 10 e no outro 20, aumentou 100%, tipo isso

In [ ]:
def calcular_taxa_mortalidade(df: pd.DataFrame) -> float:
    """
    Calcula a taxa de mortalidade.

    Args:
        df: O DataFrame do pandas com os dados dos casos.

    Returns:
        A taxa de mortalidade em porcentagem.
    """
    casos_considerados = df[df["EVOLUCAO"] != "IGNORADO"]
    obitos = (casos_considerados["EVOLUCAO"] == "OBITO").sum()
    taxa_mortalidade = (obitos / len(casos_considerados)) * 100
    return taxa_mortalidade

In [ ]:
def calcular_taxa_ocupacao_uti(df: pd.DataFrame) -> float:
    """
    Calcula a taxa de ocupação de UTI.

    Args:
        df: O DataFrame do pandas com os dados dos casos.

    Returns:
        A taxa de ocupação de UTI em porcentagem.
    """
    casos_considerados = df[df["UTI"] != "IGNORADO"]
    uti_sim = (casos_considerados["UTI"] == "SIM").sum()
    taxa_uti = (uti_sim / len(casos_considerados)) * 100
    return taxa_uti

In [ ]:
def calcular_taxas_vacinacao(df: pd.DataFrame) -> dict:
    """
    Calcula as taxas de vacinação para a gripe e COVID-19.

    Args:
        df: O DataFrame do pandas com os dados dos casos.

    Returns:
        Um dicionário contendo as taxas de vacinação para a gripe e COVID-19.
    """
    taxas = {}

    # Taxa de vacinação da gripe
    casos_vacina_gripe = df[df["VACINA"] != "IGNORADO"]
    vacinados_gripe = (casos_vacina_gripe["VACINA"] == "SIM").sum()
    taxas["gripe"] = (vacinados_gripe / len(casos_vacina_gripe)) * 100

    # Taxa de vacinação da COVID-19
    casos_vacina_covid = df[df["VACINA_COV"] != "IGNORADO"]
    vacinados_covid = (casos_vacina_covid["VACINA_COV"] == "SIM").sum()
    taxas["covid"] = (vacinados_covid / len(casos_vacina_covid)) * 100

    return taxas

In [ ]:
caminho_arquivo = "../data/03_final/dataset.csv"

# Carrega e prepara os dados
df_processado = carregar_e_preparar_dados(caminho_arquivo)

# Gera os gráficos
gerar_grafico_casos_diarios(df_processado.copy())
gerar_grafico_casos_mensais(df_processado.copy())

# Calcular e imprimir as métricas
taxa_aumento = calcular_taxa_aumento_casos(df_processado)
print("----- Taxa de Aumento de Casos (%) -----")
print(taxa_aumento)
print("\n" + "=" * 40 + "\n")

taxa_mortalidade = calcular_taxa_mortalidade(df_processado)
print(f"Taxa de Mortalidade: {taxa_mortalidade:.2f}%")
print("\n" + "=" * 40 + "\n")

taxa_uti = calcular_taxa_ocupacao_uti(df_processado)
print(f"Taxa de Ocupação de UTI: {taxa_uti:.2f}%")
print("\n" + "=" * 40 + "\n")

taxas_vacinacao = calcular_taxas_vacinacao(df_processado)
print("----- Taxas de Vacinação -----")
print(f"Gripe: {taxas_vacinacao['gripe']:.2f}%")
print(f"COVID-19: {taxas_vacinacao['covid']:.2f}%")
print("\n" + "=" * 40 + "\n")